# Exercise - GANs for image generation 

1. Train a GAN on CIFAR-10 to generate images.
1. Generate images using your trained GAN, interpolating over the latent space. Does it appear meaningful? Try such things as changing only one of the dimensions and keeping the rest fixed, trying to find an interpretation of the specific factor.
1. (Bonus): Modify your GAN to be conditional. This means that the label information should now be provided as one of the inputs for the generator and as one of the targets for the discriminator.

**Note**: You are unlikely to have time for the training to take place during class, so treat the exercise primarily as reaching the point where training is possible. Then, you can take the time at a later point. Try to solve **1** up to the point where training can start, and then use the rest as additional exercises post-lecture.

**Hint**: Consider looking at https://www.tensorflow.org/tutorials/generative/dcgan, as they go through many of the same steps.

**See slides for more details!**

# Setup

You do not have (but are of course welcome) to change any of the setup code.

Note that we use 1 to indicate "real" data and 0 to indicate "fake" data for the discriminator.

In the loss of the generator, this is "reversed", i.e. fake data is 1. This is since it needs to learn to create fake data that the generator believes is real.

In [ ]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = (x_train / 127.5) - 1, (x_test / 127.5) - 1

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

def eval_generator(generator, discriminator, fixed_noise):
    gen_predictions = generator(fixed_noise, training=False)
    
    dis_predictions = discriminator.predict(generator(tf.random.normal([256, NOISE_DIM]), training=False))
    dis_acc = round(np.mean(dis_predictions < 0.5) * 100, 2)
    
    print(f'Discriminator accuracy on fake images: {dis_acc}%.')
    
    fig = plt.figure(figsize=(4, 4))

    for i in range(gen_predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow((gen_predictions[i] + 1)/2)
        plt.axis('off')

    plt.show()

# Exercise 1

Train a GAN on CIFAR-10 to generate images.

Some settings. Feel free to change them if you want to.

In [ ]:
BATCH_SIZE = 256
NOISE_DIM = 20
fixed_noise = tf.random.normal([16, NOISE_DIM])

train_dataset = tf.data.Dataset.from_tensor_slices(x_train).shuffle(60_000).batch(BATCH_SIZE)

Let us start by defining the generator and discriminator, as well as their optimizers.

In [ ]:
generator = tf.keras.models.Sequential([
    # NOTES: Must have input shape to match noise dimension and output shape (32, 32, 3)
])
generator.summary()

discriminator = tf.keras.models.Sequential([
    # NOTES: Must have input shape (32, 32, 3) and output shape 1 (dense layer with 1 node and NO activation)
])
discriminator.summary()

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

Let us proceed by defining the training-step function we want to use.

In [ ]:
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, NOISE_DIM])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

Now, let us train the model!

In [ ]:
eval_generator(generator, discriminator, fixed_noise)

for epoch in range(40):
    for train_x in train_dataset:
        train_step(train_x)

    print(f'Epoch: {epoch}')
    eval_generator(generator, discriminator, fixed_noise)

# Exercise 2

Generate images using your trained GAN, interpolating over the latent space. Does it appear meaningful? Try such things as changing only one of the dimensions and keeping the rest fixed, trying to find an interpretation of the specific factor.

Let us generate one sample of noise, and then changing the first factor of that to produce a few examples.

In [ ]:
noise = tf.random.normal([1, NOISE_DIM]).numpy()
generated_images = generator.predict(noise)
plt.imshow((generated_images[0] + 1) / 2)

In [ ]:
noise_latent = np.concatenate([noise.copy() for _ in range(16)])
noise_latent[:, 0] = np.linspace(-3, 3, len(noise_latent))

In [ ]:
generated_images_latent = generator.predict(noise_latent)

fig = plt.figure(figsize=(4, 4))

for i in range(generated_images_latent.shape[0]):
    plt.subplot(4, 4, i+1)
    plt.imshow((generated_images_latent[i] + 1)/2)
    plt.axis('off')

plt.show()

It seems this dimension has a lot to do with color, but also some structure (see e.g. the very top of the image, goes from quite sharp to more blurry).

Let us try with other dimensions

In [ ]:
# CODE HERE

# Exercise 3

(Bonus): Modify your GAN to be conditional. This means that the label information should now be provided as one of the inputs for the generator and as one of the targets for the discriminator.